In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyecharts.options as opt
import warnings
from pyecharts.globals import CurrentConfig, NotebookType
from pypalettes import load_cmap, get_hex
from pyecharts.charts import Pie

# 忽略警告信息
warnings.filterwarnings('ignore')

# Matplotlib配置
plt.rcParams["font.family"] = 'kaiti'  # 设置中文字体为楷体
plt.rcParams['axes.unicode_minus'] = False  # 解决负号显示问题

# Pyecharts笔记本配置
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK  # 设置为Jupyter Notebook环境


ModuleNotFoundError: No module named 'pypalettes'